In [68]:
import os
import json
import time
import shutil
import winsound
import requests
import numpy as np
import pandas as pd

In [1]:
import os
import shutil

# コメントファイルの移動
comments_folder = "data/comments"
for comments_file in os.listdir(comments_folder):
    channel_name = comments_file.split("_")[0]
    folder_name = f"data/{channel_name}"
    os.makedirs(folder_name, exist_ok=True)
    comments_file_src = os.path.join(comments_folder, comments_file)
    comments_file_dst = os.path.join(folder_name, comments_file)
    shutil.move(comments_file_src, comments_file_dst)

# 動画ファイルの移動
videos_folder = "data/videos"
for videos_file in os.listdir(videos_folder):
    channel_name = videos_file.split("_")[0]
    folder_name = f"data/{channel_name}"
    os.makedirs(folder_name, exist_ok=True)
    videos_file_src = os.path.join(videos_folder, videos_file)
    videos_file_dst = os.path.join(folder_name, videos_file)
    shutil.move(videos_file_src, videos_file_dst)

In [86]:
def read_comments(file):
    with open(file, 'r', encoding='utf-8') as f:
        comments = json.load(f)
    
    data_list = []
    for c in comments:
        try:
            data = {
                'etag': c['etag'],  # コメントのetag
                'videoId': c['snippet']['videoId'],  # 動画ID
                'text': c['snippet']['topLevelComment']['snippet']['textOriginal'],  # コメントのテキスト
                'author': c['snippet']['topLevelComment']['snippet']['authorDisplayName'],  # コメント投稿者の表示名
                'authorId': c['snippet']['topLevelComment']['snippet']['authorChannelId']['value'],  # コメント投稿者のチャンネルID
                'likeCount': c['snippet']['topLevelComment']['snippet']['likeCount'],  # コメントの「いいね」数
                'ReplyCount': c['snippet']['totalReplyCount'],  # 返信コメントの数
                'date': c['snippet']['topLevelComment']['snippet']['updatedAt']  # コメントの更新日時
            }
            data_list.append(data)
        except KeyError as e:
            # エラーが発生した場合は処理をスキップ
            # print(f"エラー: {e}")
            pass
    
    df = pd.DataFrame(data_list)
    return df

def remove_duplicates(df, column):
    # 重複する要素を検索
    duplicates = df[column][df[column].duplicated()]
    
    # 重複要素をプリント
    print("重複する要素:")
    print(duplicates)
    
    # ユーザーの確認を取る
    answer = input("重複を削除しますか？(y/n): ")
    
    if answer.lower() == 'y':
        # 重複を削除
        df = df.drop_duplicates(subset=column)
        print("重複が削除されました。")
    else:
        print("重複削除はキャンセルされました。")
    
    return df

In [102]:
channel = "@quizknock" #@param{type:'string'}
file = f"data/{channel}/{channel}_comments.json"
df = read_comments(file)

FileNotFoundError: [Errno 2] No such file or directory: 'data/@quizknock/@quizknock_comments.json'

In [100]:
# 重複する要素を削除したい列の名前
column_name = 'etag'

# DataFrameから重複を削除
df = remove_duplicates(df, column_name)

重複する要素:
Series([], Name: etag, dtype: object)


重複を削除しますか？(y/n):  y


重複が削除されました。


In [101]:
destination_folder = f'data/{channel}'
df.to_json(f'{destination_folder}/{channel}_cleanComments.json', orient='records')